In [2]:
import json
import matplotlib.pyplot as plt
from collections import Counter
import statistics
from datetime import datetime
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
import re
import string

import torch
from torch import nn
import torch.nn.functional as F
from sklearn.feature_extraction.text import CountVectorizer

from collections import Counter


from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [17]:
review_train = []
with open("../dataset/review_train.json", "r") as file:
    for line in file:
        review_train.append(json.loads(line))

review_test = []
with open("../dataset/review_test.json", "r") as file:
    for line in file:
        review_test.append(json.loads(line))

In [4]:
review_train[0]
# len(review_train)

{'votes': {'funny': 0, 'useful': 1, 'cool': 0},
 'user_id': 'qVlXke5-Qm9Xr0SBcBTTnA',
 'review_id': 'K80HPQBnSg7alK7R7tIs3A',
 'stars': 1,
 'date': '2012-07-02',
 'text': 'Terrible place to shop!!! The manager is an expletive i cannot say on here. I went and bought several nails and screws as well as a rug and other items. While i was checking out the cashier charged me incorrectly for some of the screws, actually charged me more than what they were so i told the cashier and she corrected her mistake. While shopping i was there for about an hour because i was looking at paint, i even talked to a guy while i was there about different paint options as well as another lady about drill bits and about my cat condo project. anyway after i left i realize after i got home the cashier had also forgotten to give me a receipt. the rug i purchased did not work where i wanted it so i attempted to return it about 1 week after i purchased it, i went back the following weekend and the manager refused.

In [5]:
#spliting data 
train_data = []
N = 110355
for d in review_train[:N]:
    train_data.append(d)
print(len(train_data))
valid_data = []
for d in review_train[N:]:
    valid_data.append(d)
print(len(valid_data))

110355
73570


In [7]:
def process_sequence(datum):
    seq = []
    sp = set(string.punctuation)
    # wordCount=defaultdict(int)
    r = ''.join([c for c in datum['text'].lower() if not c in sp])
    seq = r.split()
    return seq 
    # print(r)
    # for w in r.split():
    #     wordCountTrain[w] += 1

In [8]:
sequences_train = []
for d in review_train:
    sequences_train.append(process_sequence(d))
    # print(process_sequence(d))
ratings_train =[]
for d in review_train:
    ratings_train.append(d['stars'])
    # print(d['stars'])
len(sequences_train)



183925

In [10]:
import numpy as np
from collections import defaultdict, Counter
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import string
from scipy.sparse import lil_matrix


class NGramRatingModel:
    def __init__(self, n, alpha=1.0):
        self.n = n
        self.alpha = alpha
        self.ngrams = defaultdict(list)
        self.context_counts = Counter()
        self.regressor = Ridge(alpha=self.alpha)

    def train(self, sequences, ratings):
        for seq, rating in zip(sequences, ratings):
            seq = ['<s>'] * (self.n - 1) + seq + ['</s>']
            for i in range(len(seq) - self.n + 1):
                ngram = tuple(seq[i:i + self.n])
                self.ngrams[ngram].append(rating)
                self.context_counts[ngram[:-1]] += 1
        self.ngrams = {ngram: ratings for ngram, ratings in self.ngrams.items() if len(ratings) > 1}
        self.ngram_to_idx = {ngram: idx for idx, ngram in enumerate(self.ngrams.keys())}

    def _extract_features_and_targets(self, sequences, ratings):
        feature_matrix = lil_matrix((len(sequences), len(self.ngrams)), dtype=np.float32)
        for i, seq in enumerate(sequences):
            feature_vector = self._extract_features(seq)
            feature_matrix[i, :] = feature_vector
        return feature_matrix.tocsr(), np.array(ratings, dtype=np.float32)

    def _extract_features(self, sequence):
        sequence = ['<s>'] * (self.n - 1) + sequence + ['</s>']
        feature_vector = np.zeros(len(self.ngrams), dtype=np.float32)
        for i in range(len(sequence) - self.n + 1):
            ngram = tuple(sequence[i:i + self.n])
            if ngram in self.ngram_to_idx:
                idx = self.ngram_to_idx[ngram]
                feature_vector[idx] += 1
        return feature_vector

    def evaluate(self, sequences, ratings):
        features, targets = self._extract_features_and_targets(sequences, ratings)
        predictions = self.regressor.predict(features)
        mse = mean_squared_error(targets, predictions)
        return mse


def process_sequence(datum):
    sp = set(string.punctuation)
    r = ''.join([c for c in datum['text'].lower() if c not in sp])
    return r.split()





In [21]:
N = len(review_train)
# Preprocessing
train_sequences = [process_sequence(d) for d in review_train[:10000]]
train_ratings = [d['stars'] for d in review_train[:10000]]

val_sequences = [process_sequence(d) for d in review_train[40000:41999]]
val_ratings = [d['stars'] for d in review_train[40000:41999]]




In [22]:
#unigram DON'T WORK TOO WELL

# Train and Evaluate
ngram_model = NGramRatingModel(n=1, alpha=0.5)
ngram_model.train(train_sequences, train_ratings)

# Train Ridge Regressor
features, targets = ngram_model._extract_features_and_targets(train_sequences, train_ratings)
ngram_model.regressor.fit(features, targets)

# Validation and Test
val_mse = ngram_model.evaluate(val_sequences, val_ratings)

print(f"Validation MSE: {val_mse:.4f}")

Validation MSE: 5.4231


In [26]:
#bigram
# Train and Evaluate
ngram_model = NGramRatingModel(n=2, alpha=0.5)
ngram_model.train(train_sequences, train_ratings)

# Train Ridge Regressor
features, targets = ngram_model._extract_features_and_targets(train_sequences, train_ratings)
ngram_model.regressor.fit(features, targets)

# Validation and Test
val_mse = ngram_model.evaluate(val_sequences, val_ratings)

print(f"Validation MSE: {val_mse:.4f}")

Validation MSE: 1.2992


In [27]:

test_sequence = [process_sequence(d) for d in review_test]
test_rating = [d['stars'] for d in review_test]

test_mse = ngram_model.evaluate(test_sequence,test_rating)
print(f"Test MSE: {test_mse:.4f}")


Test MSE: 1.3178


In [18]:
#baseline
train_average_stars = sum(d["stars"] for d in review_train) / len(review_train)
mse = sum((d["stars"] - train_average_stars) ** 2 for d in review_test) / len(review_test)
print(mse)

1.490833578439748
